In [7]:
import numpy as np
import pandas as pd
import os
import math

DAYS_IN_PAY_PERIOD = 14
WORK_HRS = 8
PAY_PERIOD_HRS = 80
AL_HRS = 8
LV_BANK_HRS = 4
MAX_CARRY_OVER = 240
ANNUAL = 0
SICK = 1
CREDIT = 2

(sun0, mon0, tue0, wed0, thu0, fri0, sat0, sun1, mon1, tue1, wed1, thu1, fri1, sat1) = range(DAYS_IN_PAY_PERIOD)
labels = ['RG', 'LA', 'LS', 'CD', 'CN', 'LH', 'LN', 'LV', 'LY']
NUM_LABELS = len(labels)
dlabels = ['sun0', 'mon0', 'tue0', 'wed0', 'thu0', 'fri0', 'sat0', 
           'sun1', 'mon1', 'tue1', 'wed1', 'thu1', 'fri1', 'sat1']
days = dict(zip(range(DAYS_IN_PAY_PERIOD), dlabels))
row_names = dict(zip(range(NUM_LABELS), labels))
row_names[NUM_LABELS] = "Total"

def print_by_lv_cat(df):
    if df.loc['RG'].sum() > 0: print(f"RG: {df.loc['RG'].sum()}") 
    if df.loc['LA'].sum() > 0: print(f"LA: {df.loc['LA'].sum()}")
    if df.loc['LS'].sum() > 0: print(f"LS: {df.loc['LS'].sum()}")
    if df.loc['CD'].sum() > 0: print(f"CD: {df.loc['CD'].sum()}")
    if df.loc['CN'].sum() > 0: print(f"CN: {df.loc['CN'].sum()}")
    if df.loc['LH'].sum() > 0: print(f"LH: {df.loc['LH'].sum()}")
    if df.loc['LN'].sum() > 0: print(f"LN: {df.loc['LN'].sum()}")
    if df.loc['LV'].sum() > 0: print(f"LV: {df.loc['LV'].sum()}")
    if df.loc['LY'].sum() > 0: print(f"LY: {df.loc['LY'].sum()}")
    print("")

def sum_by_leave_category(df):
    rgs = df.loc['RG'].sum()
    las = df.loc['LA'].sum()
    lss = df.loc['LS'].sum()
    cds = df.loc['CD'].sum()
    cns = df.loc['CN'].sum()
    lhs = df.loc['LH'].sum()
    lns = df.loc['LN'].sum()
    lvs = df.loc['LV'].sum()  # mba
    lys = df.loc['LY'].sum()  # time off award
    sums = pd.Series([rgs, las, lss, cds, cns, lhs, lns, lvs, lys], index=labels)
    return sums

def sum_by_days(df):
    day_sums = pd.Series([df['sun0'].sum(), df['mon0'].sum(), df['tue0'].sum(), df['wed0'].sum(), df['thu0'].sum(), df['fri0'].sum(), df['sat0'].sum(),
                          df['sun1'].sum(), df['mon1'].sum(), df['tue1'].sum(), df['wed1'].sum(), df['thu1'].sum(), df['fri1'].sum(), df['sat1'].sum()],
                        index=dlabels)
    return day_sums

def update(day, code, val):
    df.loc[code.upper(), days[day]] = val
                                             
def print_leave(leave, msg=''):    
    print("{0:20s}  {1:6}          {2:6}          {3:6}".format(msg, leave[ANNUAL], leave[SICK], leave[CREDIT]))
    
def print_leave_summary(leave):
    total_leave = leave[ANNUAL] + leave[CREDIT]
    al_in_days = total_leave // WORK_HRS
    sl_in_days = leave[SICK] // WORK_HRS
    al_remainder = total_leave % WORK_HRS
    sl_remainder = leave[SICK] % WORK_HRS
    print("                       {0:2} d {1} hrs  {2:2} d {3} hrs".
          format(al_in_days, al_remainder, sl_in_days, sl_remainder))
    
def print_usable_leave(leave):    
    sign = ""
    free_to_use = (leave[ANNUAL] - MAX_CARRY_OVER) + leave[CREDIT]
    if free_to_use < 0: sign = '-'
    usable_days = math.fabs(free_to_use) // WORK_HRS
    usable_hrs = math.fabs(free_to_use) % WORK_HRS
    print(f'Usable leave: {sign}{usable_days:2} d {usable_hrs}')    
    
def compute_leave(leave):    
    leave[ANNUAL] = leave[ANNUAL] + AL_HRS - df.loc['LA'].sum()
    leave[SICK] = leave[SICK] + LV_BANK_HRS - df.loc['LS'].sum()
    leave[CREDIT] = leave[CREDIT] + df.loc['CD'].sum() - df.loc['CN'].sum()
    return leave                                                     
                     
def verify80(leave):
    total_hrs = 0.0
    for label in labels:
        total_hrs = total_hrs + df.loc[label].sum()
    expected80 = total_hrs - df.loc['CD'].sum()
    if (expected80 != PAY_PERIOD_HRS):
        print("wARNING: leave earned not equal to 80.0")
                                     
def print_time_sheet(df, leave):                                          
    df_rowsum = pd.concat([df, sum_by_leave_category(df)], axis=1)
    df_rowsum.rename(columns={0:'Total'})
    df_rowsum2 = df_rowsum.rename(columns={0:'Total'})

    df_colsum = df_rowsum2.append(sum_by_days(df),ignore_index=True)
    df_colsum.rename(index=row_names, inplace=True)

    total_leave = sum_by_leave_category(df).sum()
    total_days = sum_by_days(df).sum()
    remaining_hrs = PAY_PERIOD_HRS - total_days + df.loc['CD'].sum()

    #print("\n")
    print_by_lv_cat(df)
    print(f"total by leave: {total_leave}")
    print(f"total by day:   {total_days}")
    print(f"remaining to work:   {remaining_hrs}")                  
    verify80(df)                                     
    essential = df_colsum.loc[(df_colsum!=0).any(1)]
    print()
    print(f'Leave Balance:         A/L             S/L              Credit')
    print("---------------------------------------------------------------")
    print_leave(leave, "Start of Pay Period:")
    print_leave_summary(leave)
    leave = compute_leave(leave)
    print()
    print_leave(leave, "End of Pay Perod:")
    print_leave_summary(leave)
    print("\n")
    print_usable_leave(leave)
    return essential
                                     
def update1(theDay, *vals):
    for i in range(0, len(vals), 2):
        df.loc[vals[i].upper(), days[theDay]] = vals[i+1]

In [21]:
## Default Start for Pay Period
# times = np.zeros((NUM_LABELS, 14))
# df = pd.DataFrame(times, index=labels, columns=dlabels)
# leave = [000.0, 000.0, 0.0]
# update1(mon0, 'RG', 8)
# update1(tue0, 'RG', 8)
# update1(wed0, 'RG', 8)
# update1(thu0, 'RG', 8)
# update1(fri0, 'RG', 8)
# update1(mon1, 'RG', 8)
# update1(tue1, 'RG', 8)
# update1(wed1, 'RG', 8)
# update1(thu1, 'RG', 8)
# update1(fri1, 'RG', 8)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'J2020' (Current)

In [22]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [234.0, 246.0, 24.0]
update1(mon0, 'RG', 8, 'LN', 0.5)
update1(tue0, 'RG', 8)
update1(wed0, 'RG', 8)
update1(thu0, 'RG', 8)
update1(fri0, 'RG', 8)
update1(mon1, 'RG', 8)
update1(tue1, 'RG', 8)
update1(wed1, 'RG', 8)
update1(thu1, 'RG', 8)
update1(fri1, 'RG', 7.5)
essential = print_time_sheet(df, leave)
essential = essential.replace(0.0, '-')
essential

RG: 79.5
LN: 0.5

total by leave: 80.0
total by day:   80.0
remaining to work:   0.0

Leave Balance:         A/L             S/L              Credit
---------------------------------------------------------------
Start of Pay Period:   234.0           246.0            24.0
                       32.0 d 2.0 hrs  30.0 d 6.0 hrs

End of Pay Perod:      242.0           250.0            24.0
                       33.0 d 2.0 hrs  31.0 d 2.0 hrs


Usable leave: 3.0 d 2.0


,sun0,mon0,tue0,wed0,thu0,fri0,sat0,sun1,mon1,tue1,wed1,thu1,fri1,sat1,Total
RG,-,8.0,8,8,8,8,-,-,8,8,8,8,7.5,-,79.5
LN,-,0.5,-,-,-,-,-,-,-,-,-,-,-,-,0.5
Total,-,8.5,8,8,8,8,-,-,8,8,8,8,7.5,-,NaN


### Pay Period 'I2020'

In [17]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [226.0, 242.0, 24.0]
update1(mon0, 'RG', 8.5, 'LN', 0.5)
update1(tue0, 'RG', 8.5, 'LN', 0.5)
update1(wed0, 'RG', 8.0, 'LN', 0.5)
update1(thu0, 'RG', 8.5, 'LN', 1.0)
update1(fri0, 'RG', 7.5, 'LN', 0.5)
update1(mon1, 'RG', 8.0, 'LN', 0.5)
update1(tue1, 'RG', 8.0, 'LN', 0.5)
update1(wed1, 'RG', 9.0, 'LN', 0.5)
update1(thu1, 'RG', 8.0, 'LN', 0.5)
update1(fri1, 'RG', 0.5, 'LN', 0.5)
essential = print_time_sheet(df, leave)
essential = essential.replace(0.0, '-')
essential

RG: 74.5
LN: 5.5

total by leave: 80.0
total by day:   80.0
remaining to work:   0.0

Leave Balance:         A/L             S/L              Credit
---------------------------------------------------------------
Start of Pay Period:   226.0           242.0            24.0
                       31.0 d 2.0 hrs  30.0 d 2.0 hrs

End of Pay Perod:      234.0           246.0            24.0
                       32.0 d 2.0 hrs  30.0 d 6.0 hrs


Usable leave: 2.0 d 2.0


,sun0,mon0,tue0,wed0,thu0,fri0,sat0,sun1,mon1,tue1,wed1,thu1,fri1,sat1,Total
RG,-,8.5,8.5,8.0,8.5,7.5,-,-,8.0,8.0,9.0,8.0,0.5,-,74.5
LN,-,0.5,0.5,0.5,1.0,0.5,-,-,0.5,0.5,0.5,0.5,0.5,-,5.5
Total,-,9.0,9.0,8.5,9.5,8.0,-,-,8.5,8.5,9.5,8.5,1.0,-,NaN


### Pay Period 'H2020'

In [18]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [218.0, 240.0, 20.0]
update1(mon0, 'RG', 8.5, 'LN', 0.5)
update1(tue0, 'RG', 7.5, 'LN', 1.0)
update1(wed0, 'RG', 8.5, 'LN', 0.5)
update1(thu0, 'RG', 8.0, 'LN', 0.5)
update1(fri0, 'RG', 5.5, 'LN', 0.5, 'LS', 2.0)
update1(mon1, 'RG', 7.5, 'LN', 1.0)
update1(tue1, 'RG', 8.5, 'LN', 0.5)
update1(wed1, 'RG', 8.5, 'LN', 0.5)
update1(thu1, 'RG', 8.0, 'LN', 0.5)
update1(fri1, 'RG', 1.5, 'LN', 0.5, 'CD', 4.0)
essential = print_time_sheet(df, leave)
essential = essential.replace(0.0, '-')
essential

RG: 72.0
LS: 2.0
CD: 4.0
LN: 6.0

total by leave: 84.0
total by day:   84.0
remaining to work:   0.0

Leave Balance:         A/L             S/L              Credit
---------------------------------------------------------------
Start of Pay Period:   218.0           240.0            20.0
                       29.0 d 6.0 hrs  30.0 d 0.0 hrs

End of Pay Perod:      226.0           242.0            24.0
                       31.0 d 2.0 hrs  30.0 d 2.0 hrs


Usable leave: 1.0 d 2.0


,sun0,mon0,tue0,wed0,thu0,fri0,sat0,sun1,mon1,tue1,wed1,thu1,fri1,sat1,Total
RG,-,8.5,7.5,8.5,8,5.5,-,-,7.5,8.5,8.5,8,1.5,-,72.0
LS,-,-,-,-,-,2,-,-,-,-,-,-,-,-,2.0
CD,-,-,-,-,-,-,-,-,-,-,-,-,4,-,4.0
LN,-,0.5,1,0.5,0.5,0.5,-,-,1,0.5,0.5,0.5,0.5,-,6.0
Total,-,9,8.5,9,8.5,8,-,-,8.5,9,9,8.5,6,-,NaN


### Pay Period 'G2020'

In [19]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [210.0, 236.0, 14.0]
update1(mon0, 'RG', 8, 'LN', 1)
update1(tue0, 'RG', 7.5, 'LN', 0.5)
update1(wed0, 'RG', 7.5, 'LN', 0.5)
update1(thu0, 'RG', 8.5, 'LN', 0.5)
update1(fri0, 'RG', 7.5, 'LN', 0.5)
update1(mon1, 'RG', 9, 'LN', 1)
update1(tue1, 'RG', 9, 'LN', 0.5)
update1(wed1, 'RG', 8.5, 'LN', 0.5)
update1(thu1, 'RG', 8, 'LN', 0.5)
update1(fri1, 'RG', 0.5, 'LN', 0.5, 'CD', 6)
essential = print_time_sheet(df, leave)
essential = essential.replace(0.0, '-')
essential

RG: 74.0
CD: 6.0
LN: 6.0

total by leave: 86.0
total by day:   86.0
remaining to work:   0.0

Leave Balance:         A/L             S/L              Credit
---------------------------------------------------------------
Start of Pay Period:   210.0           236.0            14.0
                       28.0 d 0.0 hrs  29.0 d 4.0 hrs

End of Pay Perod:      218.0           240.0            20.0
                       29.0 d 6.0 hrs  30.0 d 0.0 hrs


Usable leave: -0.0 d 2.0


,sun0,mon0,tue0,wed0,thu0,fri0,sat0,sun1,mon1,tue1,wed1,thu1,fri1,sat1,Total
RG,-,8,7.5,7.5,8.5,7.5,-,-,9,9,8.5,8,0.5,-,74.0
CD,-,-,-,-,-,-,-,-,-,-,-,-,6.0,-,6.0
LN,-,1,0.5,0.5,0.5,0.5,-,-,1,0.5,0.5,0.5,0.5,-,6.0
Total,-,9,8,8,9,8,-,-,10,9.5,9,8.5,7.0,-,NaN


### Pay Period 'F2020'

In [4]:
## Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [202.0, 232.0, 9.0]
update1(mon0, 'RG', 8, 'LN', 1.0)
update1(tue0, 'RG', 9)
update1(wed0, 'RG', 8.5, 'LN', 0.5)
update1(thu0, 'RG', 8, 'LN', 1)
update1(fri0, 'RG', 7.5, 'LN', 0.5)
update1(mon1, 'RG', 9)
update1(tue1, 'RG', 7, 'LN', 1)
update1(wed1, 'RG', 8, 'LN', 0.5)
update1(thu1, 'RG', 7.5, 'LN', 0.5)
update1(fri1, 'RG', 1.5, 'CD', 5, 'LN', 1.0)
essential = print_time_sheet(df, leave)
essential = essential.replace(0.0, '-')
essential

RG: 74.0
CD: 5.0
LN: 6.0

total by leave: 85.0
total by day:   85.0
remaining to work:   0.0

Leave Balance:         A/L             S/L              Credit
---------------------------------------------------------------
Start of Pay Period:   202.0           232.0             9.0
                       26.0 d 3.0 hrs  29.0 d 0.0 hrs

End of Pay Perod:      210.0           236.0            14.0
                       28.0 d 0.0 hrs  29.0 d 4.0 hrs


Usable leave: -2.0 d 0.0


,sun0,mon0,tue0,wed0,thu0,fri0,sat0,sun1,mon1,tue1,wed1,thu1,fri1,sat1,Total
RG,-,8,9,8.5,8,7.5,-,-,9,7,8,7.5,1.5,-,74.0
CD,-,-,-,-,-,-,-,-,-,-,-,-,5.0,-,5.0
LN,-,1,-,0.5,1,0.5,-,-,-,1,0.5,0.5,1.0,-,6.0
Total,-,9,9,9,9,8,-,-,9,8,8.5,8,7.5,-,NaN


### Pay Period 'E2020'

In [20]:
## Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [194.0, 251.0, 6.0]
update1(mon0, 'RG', 4, 'LS', 4.0)
update1(tue0, 'RG', 8.5, 'LN', 0.5)
update1(wed0, 'RG', 8.0, 'LN', 1.0)
update1(thu0, 'RG', 1.0, 'LS', 7.0)
update1(fri0, 'RG', 0.0, 'LS', 8.0)
update1(mon1, 'RG', 7.5, 'LN', 1.0)
update1(tue1, 'RG', 6.0, 'LS', 2.0)
update1(wed1, 'RG', 7.5, 'LN', 1.0)
update1(thu1, 'RG', 8)
update1(fri1, 'RG', 3.0, 'CD', 3.0, 'LS', 2.0)
essential = print_time_sheet(df, leave)
essential = essential.replace(0.0, '-')
essential

RG: 53.5
LS: 23.0
CD: 3.0
LN: 3.5

total by leave: 83.0
total by day:   83.0
remaining to work:   0.0

Leave Balance:         A/L             S/L              Credit
---------------------------------------------------------------
Start of Pay Period:   194.0           251.0             6.0
                       25.0 d 0.0 hrs  31.0 d 3.0 hrs

End of Pay Perod:      202.0           232.0             9.0
                       26.0 d 3.0 hrs  29.0 d 0.0 hrs


Usable leave: -3.0 d 5.0


,sun0,mon0,tue0,wed0,thu0,fri0,sat0,sun1,mon1,tue1,wed1,thu1,fri1,sat1,Total
RG,-,4,8.5,8,1,-,-,-,7.5,6,7.5,8,3,-,53.5
LS,-,4,-,-,7,8,-,-,-,2,-,-,2,-,23.0
CD,-,-,-,-,-,-,-,-,-,-,-,-,3,-,3.0
LN,-,-,0.5,1,-,-,-,-,1,-,1,-,-,-,3.5
Total,-,8,9,9,8,8,-,-,8.5,8,8.5,8,8,-,NaN


### Pay Period 'D2020'

In [37]:
## Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [186.0, 256.5, 0.0]
update1(mon0, 'LH', 8)
update1(tue0, 'RG', 8.5, 'LN', 0.5)
update1(wed0, 'RG', 8.0, 'LN', 1.0)
update1(thu0, 'RG', 9.5)
update1(fri0, 'RG', 5.5, 'LS', 1.5, 'LN', 1.0)
update1(mon1, 'RG', 7.5, 'LN', 1.0)
update1(tue1, 'RG', 4.5, 'LS', 3.5)
update1(wed1, 'RG', 3.5, 'LS', 4.5)
update1(thu1, 'RG', 10)
update1(fri1, 'RG', 1, 'LN', 1.0, 'CD', 6)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'C2020'

In [60]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [250.0, 252.5, 0.0]
update1(mon0, 'LY', 8)
update1(tue0, 'LA', 8)
update1(wed0, 'LA', 8)
update1(thu0, 'LA', 8)
update1(fri0, 'LA', 8)
update1(mon1, 'LA', 8)
update1(tue1, 'LA', 8)
update1(wed1, 'LA', 8)
update1(thu1, 'LA', 8)
update1(fri1, 'LA', 8)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'B2020'

In [4]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
# removed 4 hrs of A/L for leave bank
leave = [242.0, 248.5, 7.0]
update1(mon0, 'LH', 8)
update1(tue0, 'CN', 7.0)
update1(wed0, 'RG', 8.5, 'LN', 1)
update1(thu0, 'RG', 8.0, 'LN', 0.5)
update1(fri0, 'RG', 8)
update1(mon1, 'RG', 8.0, 'LN', 1.0)
update1(tue1, 'RG', 8.5)
update1(wed1, 'RG', 7.5, 'LN', 1.0)
update1(thu1, 'RG', 8.5, 'LN', 0.5)
update1(fri1, 'RG', 4.0)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'A2020'

In [8]:
## Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [238.0, 246.5, 0.0]
update1(mon0, 'RG', 8.0, 'LN', 1.0)
update1(tue0, 'RG', 8.5)
update1(wed0, 'RG', 8.0, 'LN', 1.0)
update1(thu0, 'RG', 8.5)
update1(fri0, 'RG', 7.0, 'LN', 1.0)
update1(mon1, 'RG', 8.0, 'LN', 1.0)
update1(tue1, 'RG', 6, 'LS', 2)
update1(wed1, 'RG', 8.5, 'LN', 1.0)
update1(thu1, 'RG', 9.5)
update1(fri1, 'RG', 0.0, 'CD', 7.0, 'LN', 1.0)
essential = print_time_sheet(df, leave)
essential = essential.replace(0.0, '-')
essential

RG: 72.0
LS: 2.0
CD: 7.0
LN: 6.0

total by leave: 87.0
total by day:   87.0
remaining to work:   0.0

Leave Balance:         A/L             S/L              Credit
---------------------------------------------------------------
Start of Pay Period:   238.0           246.5             0.0
                       29.0 d 6.0 hrs  30.0 d 6.5 hrs

End of Pay Perod:      246.0           248.5             7.0
                       31.0 d 5.0 hrs  31.0 d 0.5 hrs


Usable leave: 1.0 d 5.0


,sun0,mon0,tue0,wed0,thu0,fri0,sat0,sun1,mon1,tue1,wed1,thu1,fri1,sat1,Total
RG,-,8,8.5,8,8.5,7,-,-,8,6,8.5,9.5,-,-,72.0
LS,-,-,-,-,-,-,-,-,-,2,-,-,-,-,2.0
CD,-,-,-,-,-,-,-,-,-,-,-,-,7,-,7.0
LN,-,1,-,1,-,1,-,-,1,-,1,-,1,-,6.0
Total,-,9,8.5,9,8.5,8,-,-,9,8,9.5,9.5,8,-,NaN


### Pay Period 'Z2019'

In [17]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [276.0, 242.5, 8.5]
update1(mon0, 'RG', 2.5, 'LN', 1, 'CN', 4.5)
update1(tue0, 'LN', 8)
update1(wed0, 'LH', 8)
update1(thu0, 'LA', 8)
update1(fri0, 'LA', 8)
update1(mon1, 'LA', 8)
update1(tue1, 'LA', 8)
update1(wed1, 'LH', 8)
update1(thu1, 'LA', 8)
update1(fri1, 'LA', 4, 'CN', 4.0)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'Y'

In [9]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [276.0, 245.0, 17]
update1(mon0, 'LA', 8)
update1(tue0, 'RG', 6.5, 'LV', 3)
update1(wed0, 'RG', 8.0, 'LN', 1)
update1(thu0, 'RG', 7.0, 'LV', 3)
update1(fri0, 'RG', 8.0, 'LN', 1)
update1(sun1, 'RG', 2.0)
update1(mon1, 'RG', 5.5, 'LS', 2.5)
update1(tue1, 'CN', 8.0)
update1(wed1, 'CN', 0.5)
update1(thu1, 'RG', 7.0, 'LN', 1)
update1(fri1, 'RG', 4.0, 'LS', 4)
#essential = print_time_sheet(df, leave)
#essential = essential.replace(0.0, '-')
#essential

### Pay Period 'X'

In [16]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [292.0, 241.0, 8.0]
update1(mon0, 'RG', 9.5, 'LN', 1)
update1(tue0, 'RG', 8.5, 'LN', 1)
update1(wed0, 'LA', 8)
update1(thu0, 'LH', 8)
update1(fri0, 'LA', 8)
update1(mon1, 'LA', 8)
update1(tue1, 'RG', 8.5, 'LN', 1)
update1(wed1, 'RG', 8.5, 'LN', 1)
update1(thu1, 'RG', 8, 'CD', 2.0)
update1(fri1, 'CD', 7, 'LN', 1)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'W'

In [27]:
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [292.0, 237.0, 4.5]
update(mon0, 'LH', 8)
update(tue0, 'RG', 8.5)
update(tue0, 'LN', 1)
update(wed0, 'RG', 9.5)
update(thu0, 'RG', 7.5)
update(thu0, 'LN', 1)
update(fri0, 'RG', 7)
update(fri0, 'LN', 1)
update(mon1, 'LA', 8)
update(tue1, 'RG', 8.5)
update(tue1, 'LN', 1)
update(wed1, 'RG', 9)
update(wed1, 'LN', 0.5)
update(thu1, 'RG', 8)
update(thu1, 'LN', 1)
update(fri1, 'RG', 0.0)
update(fri1, 'LN', 0.5)
update(fri1, 'CD', 3.5)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'V'

In [17]:
## Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [284.0, 234.0, 3.0]
update(mon0, 'RG', 8)
update(tue0, 'RG', 7.5)
update(tue0, 'LN', 1)
update(wed0, 'RG', 8)
update(wed0, 'LN', 0.5)
update(thu0, 'RG', 8.5)
update(fri0, 'RG', 7.5)
update(mon1, 'RG', 7)
update(mon1, 'LS', 1)
update(tue1, 'RG', 7)
update(tue1, 'LN', 1)
update(wed1, 'RG', 8)
update(thu1, 'RG', 7.5)
update(thu1, 'LN', 1)
update(fri1, 'RG', 5.5)
update(fri1, 'LN', 1)
update(fri1, 'CD', 1.5)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'U'

In [2]:
## Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [276.0, 230.0, 0.0]
update(mon0, 'LH', 8)
update(tue0, 'RG', 7.5)
update(tue0, 'LN', 1)
update(wed0, 'RG', 7.5)
update(wed0, 'LN', 1)
update(thu0, 'RG', 8.5)
update(fri0, 'RG', 7)
update(fri0, 'LN', 1)
update(mon1, 'RG', 7.5)
update(mon1, 'LN', 1)
update(tue1, 'RG', 9)
update(wed1, 'RG', 7.5)
update(wed1, 'LN', 1)
update(thu1, 'RG', 8.5)
update(fri1, 'RG', 3)
update(fri1, 'CD', 3)
update(fri1, 'LN', 1)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

### Pay Period 'T'

In [26]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [273.0, 226.0, 0.0]
update(mon0, 'RG', 7.5)
update(mon0, 'LN', 1)
update(tue0, 'RG', 4)
update(tue0, 'LV', 4)
update(wed0, 'RG', 7)
update(wed0, 'LN', 1)
update(thu0, 'RG', 9.0)
update(fri0, 'RG', 7)
update(fri0, 'LN', 1)
update(mon1, 'RG', 7.5)
update(mon1, 'LN', 1)
update(tue1, 'RG', 8.5)
update(wed1, 'RG', 8.5)
update(thu1, 'RG', 7)
update(thu1, 'LN', 1)
update(fri1, 'RG', 0)
update(fri1, 'LA', 5)
#essential = print_time_sheet(df, leave)
#essential = essential.replace(0.0, '-')
#essential

### Pay Period 'S'

In [25]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [305.0, 222.0, 0.0]
update(mon0, 'LA', 8)
update(tue0, 'LA', 8)
update(wed0, 'LA', 8)
update(thu0, 'LA', 8)
update(fri0, 'LA', 8)
update(mon1, 'RG', 7.5)
update(mon1, 'LN', 1)
update(tue1, 'RG', 8.5)
update(wed1, 'RG', 7.5)
update(wed1, 'LN', 1.0)
update(thu1, 'RG', 8.5)
update(fri1, 'RG', 5)
update(fri1, 'LN', 1)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

#### Pay Period 'R'

In [24]:
## Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [340.0, 218.0, 2.0]
update(mon0, 'LH', 8)
update(tue0, 'RG', 7.5)
update(tue0, 'LN', 1)
update(wed0, 'RG', 9)
update(wed0, 'LN', 1)
update(thu0, 'RG', 7.5)
update(thu0, 'LN', 1)
update(fri0, 'CN', 2)
update(fri0, 'LA', 3)
update(mon1, 'LA', 8)
update(tue1, 'LA', 8)
update(wed1, 'LA', 8)
update(thu1, 'LA', 8)
update(fri1, 'LA', 8)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

#### Pay Period 'Q'

In [23]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [332.0, 214.0, 18.0]
update(mon0, 'RG', 7)
update(mon0, 'LN', 1)
update(tue0, 'RG', 8.5)
update(wed0, 'RG', 8.5)
update(wed0, 'LN', 1)
update(thu0, 'RG', 7.5)
update(thu0, 'LN', 1)
update(fri0, 'CN', 8)
update(mon1, 'CN', 8)
update(tue1, 'RG', 7.5)
update(tue1, 'LN', 1)
update(wed1, 'RG', 7.5)
update(wed1, 'LN', 1)
update(thu1, 'RG', 7)
update(thu1, 'LN', 1)
update(fri1, 'RG', 4.5)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

#### Pay Period 'P'

In [22]:
## Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [324.0, 250.0, 13.0]
update(mon0, 'RG', 8)
update(mon0, 'LN', 1)
update(tue0, 'RG', 8.5)
update(wed0, 'RG', 8.0)
update(wed0, 'LN', 1)
update(thu0, 'RG', 10.0)
update(fri0, 'RG', 2.5)
update(fri0, 'CD', 5.0)
update(fri0, 'LN', 1)
update(mon1, 'LS', 8)
update(tue1, 'LS', 8)
update(wed1, 'LS', 8)
update(thu1, 'LS', 8)
update(fri1, 'LS', 8)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

#### Pay Period 'O'

In [21]:
# Default Start for Pay Period
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [316.0, 250.0, 6.5]
update(mon0, 'RG', 7)
update(mon0, 'LN', 1)
update(tue0, 'RG', 8.5)
update(wed0, 'RG', 5)
update(wed0, 'LS', 3)
update(thu0, 'RG', 8)
update(thu0, 'LN', 1)
update(fri0, 'RG', 8)
update(fri0, 'LN', 1)
update(mon1, 'RG', 8)
update(mon1, 'LN', 1)
update(tue1, 'RG', 9)
update(wed1, 'RG', 10.5)
update(wed1, 'LN', 0.5)
update(thu1, 'RG', 7)
update(thu1, 'LS', 1)
update(fri1, 'RG', 0)
update(fri1, 'LN', 0.5)
update(fri1, 'CD', 6.5)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

#### Pay Period "N"

In [20]:
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [308.0, 248.0, 0.0]
update(mon0, 'RG', 8.5)
update(tue0, 'RG', 8.0)
update(tue0, 'LN', 1)
update(wed0, 'RG', 8.0)
update(wed0, 'LN', 1)
update(thu0, 'RG', 8)
update(fri0, 'RG', 8.0)
update(fri0, 'LN', 1)
update(mon1, 'RG', 7.5)
update(mon1, 'LN', 1)
update(tue1, 'RG', 8.5)
update(wed1, 'RG', 8.0)
update(wed1, 'LN', 1.0)
update(thu1, 'RG', 8.5)
update(thu1, 'CD', 0.5)
update(fri1, 'CD', 6)
update(fri1, 'LS', 2)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

#### Pay Period "M"

In [19]:
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [315.0, 244.0, 11.5]
update(mon0, 'RG', 10.5)
update(tue0, 'RG', 8.0)
update(tue0, 'LN', 1.0)
update(wed0, 'RG', 9)
update(thu0, 'RG', 8)
update(thu0, 'LN', 1)
update(fri0, 'RG', 8)
update(mon1, 'CN', 8)
update(tue1, 'CN', 3.5)
update(wed1, 'LA', 8.0)
update(thu1, 'LH', 8)
update(fri1, 'LA', 7.0)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential

#### Pay Period "L"

In [18]:
times = np.zeros((NUM_LABELS, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)
leave = [307.0, 240.0, 0.0]
update(mon0, 'RG', 8.5)
update(mon0, 'LN', 1.0)
update(tue0, 'RG', 9.0)
update(wed0, 'RG', 8.5)
update(wed0, 'LN', 1.0)
update(thu0, 'RG', 9.5)
update(fri0, 'RG', 7.5)
update(fri0, 'LN', 1.0)
update(mon1, 'RG', 9.0)
update(tue1, 'RG', 8.0)
update(tue1, 'LN', 1.0)
update(wed1, 'RG', 10.0)
update(wed1, 'LN', 0.5)
update(thu1, 'RG', 5.5)
update(thu1, 'CD', 3.5)
update(fri1, 'CD', 8.0)
# essential = print_time_sheet(df, leave)
# essential = essential.replace(0.0, '-')
# essential